In [0]:
%run ./00_Setup

In [0]:
%sql
-- Set the working schema
USE SCHEMA strata_lab;
-- Create a text widget for the book store path
CREATE WIDGET TEXT book_store_path DEFAULT '/Volumes/workspace/strata_lab/entrenamiento/book_store';

In [0]:
# Retrieve the book_store_path value from the widget
book_store_path = dbutils.widgets.get("book_store_path")

In [0]:
# Read the 'books' table as a streaming source and write to an in-memory table
(
  spark.readStream
    .table("books")
    .writeStream
    .format("memory")
    .queryName("books_streaming_tmp_vw")
    .option("checkpointLocation", book_store_path + '/books_checkpoint')
    .trigger(availableNow=True)
    .start()
)

In [0]:
%sql
-- Query all records from the books table
SELECT * FROM books

Streaming table simulation in a 2 step python code, free edition didnt allowed me to make it as one sql query

In [0]:
%sql
-- Aggregate total books per author from the streaming view
-- Order by is not allowed in streaming dataframes
SELECT author, 
count(book_id) AS total_books 
FROM books_streaming_tmp_vw
GROUP BY author

In [0]:
%sql
-- Create a temporary view with author and their total book counts
CREATE OR REPLACE TEMP VIEW author_counts_tmp_vw AS(
  SELECT  
    author,
    count(book_id) AS total_books
  FROM books_streaming_tmp_vw
  GROUP BY author
)

In [0]:
%sql
-- Query all records from the author counts temporary view
SELECT * FROM author_counts_tmp_vw